# Reproduction of the winner algorithm for ISMIR 2004

This notebook tries to reproduce the approach of the winner algorithm for ISMIR 2004, here are the main sections:

1. [Importing modules and some constants](#imports)
2. [Loading the dataset](#loading_dataset)
3. [Processing files](#processing_files)
4. [Evaluating the accuracy](#evaluation)
5. [Showing the results](#showing_results)

<br>

If you want so see in detail how each file is processed you can take a look at this [Single file analysis notebook](single_file_analysis.ipynb)

#### Importing modules and some constants
<a id="imports"></a>

In [1]:
import sys
sys.path.append('../')

from utils.constants import *
from utils.eval_func import *
from utils.process import *
from utils.utils import *
from utils.data_management import *


#### Loading the dataset
<a id="loading_dataset"></a>

In [2]:
files = load_dataset() #list with the file names without the extension

#### Processing files
<a id="processing_files"></a>

The files are processed and the result is saved into csv files

In [3]:
for i,filename in enumerate(files):
    print("{0}/{1}".format(i+1,len(files))) #progress
    audio = load_file(filename) #loads the audio and applies an equal loudness filter
    pitch = process_audio(audio) #returns the predominant pitch per frame
    save_pitch(pitch,filename) #saves the pitches to the csv

1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20
20/20


#### Evaluating the accuracy
<a id="evaluation"></a>

In [4]:
if os.path.exists(ERROR_CSV_FILE): os.remove(ERROR_CSV_FILE)

for filename in files:
    compute_error(filename)

calculate_average_tsv(ERROR_CSV_FILE)
calculate_difference_with_reference(ERROR_CSV_FILE, ERROR_WINNER_CSV_FILE)

             evalOption1  evalOption2    Average
daisy1         21.076346    21.974818  21.525582
daisy2         11.574195    11.722405  11.648300
daisy3        -31.354342   -26.031283 -28.692812
daisy4        -20.272232   -14.822321 -17.547277
jazz1           3.839441     4.771469   4.305455
jazz2          -7.199219    -4.997685  -6.128452
jazz3          22.170908    25.811593  23.991250
jazz4          -6.248834     0.181844  -3.033495
midi1         -39.789237   -28.220909 -34.000073
midi2          -2.183133    -2.056784  -2.119959
midi3          -8.080932    -7.145828  -7.613380
midi4         -18.045172   -11.156877 -14.596025
opera_fem2     27.382514    30.646524  29.009519
opera_fem4     21.353859    22.332661  21.843260
opera_male3     8.075789    12.477019  10.276404
opera_male5    -2.611910     9.384668   3.386379
pop1           -9.786861    -9.363589  -9.575225
pop2          -11.247099    -6.937817  -9.092458
pop3           -4.513677    -2.881501  -3.692589
pop4            0.60